# Beta expanded ensemble

Example expanded ensemble simulation inverse temperature.

In [1]:
params = {"beta_min": 0.8, "beta_max": 1.2}
beta_num = 5
params["delta_beta"] = (params["beta_max"] - params["beta_min"])/(beta_num-1)

script="""
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /feasst/particle/lj.fstprt
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
ThermoParams beta 1.2 chemical_potential0 1
FlatHistogram Macrostate MacrostateBeta width {delta_beta} max {beta_max} min {beta_min} \
    Bias WLTM collect_flatness 18 min_flatness 22 min_sweeps 10
TrialTranslate weight 1 tunable_param 1
TrialAdd particle_type 0
Run until_num_particles 10
RemoveTrial name TrialAdd
TrialBeta fixed_beta_change {delta_beta}
set_variable trials_per 1e4
Log trials_per_write trials_per file_name lj_beta.txt
Movie trials_per_write trials_per file_name lj_beta.xyz
CheckEnergy trials_per_update trials_per
Tune
CriteriaUpdater trials_per_update trials_per
CriteriaWriter trials_per_write trials_per file_name lj_beta_crit.txt file_name_append_phase true
Energy file_name lj_beta_energy file_name_append_phase true trials_per_update 1 \
    trials_per_write trials_per multistate true
Run num_trials 1e6
""".format(**params)

with open('script.txt', 'w') as file: file.write(script)
import subprocess
syscode = subprocess.call("../../../build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)

# FEASST version: v0.21-21-g12898a9d3b-dirty-hwh/trial_library
Configuration cubic_side_length 8 particle_type0 /home/hwh/feasst/particle/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 1.2 chemical_potential0 1  
FlatHistogram Bias WLTM Macrostate MacrostateBeta collect_flatness 18 max 1.2 min 0.8 min_flatness 22 min_sweeps 10 width 0.09999999999999998  
TrialTranslate tunable_param 1 weight 1  
TrialAdd particle_type 0  
Run until_num_particles 10  
# initializing random number generator with seed: 1678985889
RemoveTrial name TrialAdd  
TrialBeta fixed_beta_change 0.09999999999999998  
Log file_name lj_beta.txt trials_per_write 1e4  
Movie file_name lj_beta.xyz trials_per_write 1e4  
CheckEnergy trials_per_update 1e4  
Tune  
CriteriaUpdater trials_per_update 1e4  
CriteriaWriter file_name lj_beta_crit.txt file_name_append_phase true trials_per_write 1e4  
Energy file_name lj_beta_energy file_name_append_phase true multistate t

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!